In [1]:
import pprint
from pydantic import BaseModel

def mprint(x):
    if isinstance(x, BaseModel):
        pprint.pprint(x.dict(exclude_unset=True), sort_dicts=False)
    else:
        pprint.pprint(x)

In [2]:
from service_capacity_modeling.interface import CapacityDesires
from service_capacity_modeling.interface import FixedInterval, Interval
from service_capacity_modeling.interface import QueryPattern, DataShape

desires = CapacityDesires(
    # This service is critical to the business
    service_tier=1,
    query_pattern=QueryPattern(
        # Not sure exactly how much QPS we will do, but we think around
        # 10,000 reads and 10,000 writes per second.
        estimated_read_per_second=Interval(
            low=1_000, mid=10_000, high=100_000, confidence=0.98
        ),
        estimated_write_per_second=Interval(
            low=1_000, mid=10_000, high=100_000, confidence=0.98
        ),
    ),
    # Not sure how much data, but we think it'll be around 100 GiB
    data_shape=DataShape(
        estimated_state_size_gib=Interval(low=10, mid=100, high=1_000, confidence=0.98),
    ),
)

In [3]:
from service_capacity_modeling.capacity_planner import planner
from service_capacity_modeling.models.org import netflix

# Load up the Netflix capacity models
planner.register_group(netflix.models)

plan = planner.plan(
    model_name="org.netflix.cassandra",
    region="us-east-1",
    desires=desires,
    explain=True
)

Loading shape=aws from /home/jolynch/Hacking/service-capacity-modeling/service_capacity_modeling/hardware/profiles/shapes/aws
Loading /home/jolynch/Hacking/service-capacity-modeling/service_capacity_modeling/hardware/profiles/pricing/aws/3yr-reserved.json


In [4]:
worlds = plan.explanation.regret_clusters_by_model["org.netflix.cassandra"]

def summarize(cluster, regret):
    cost = cluster.candidate_clusters.total_annual_cost
    zonal = cluster.candidate_clusters.zonal
    count = len(zonal) * zonal[0].count
    instance = zonal[0].instance.name
    print(f"{count:>3} {instance:>10} costing {cost} -> {regret}")

for cluster, regret in worlds:
    summarize(cluster, regret)

 12 m5d.xlarge costing 8945.71 -> 9818842.386522133
 12 m5d.xlarge costing 8941.18 -> 9821648.404682202
 12 m5d.xlarge costing 8940.92 -> 9821849.290123086
 12 m5d.xlarge costing 8940.67 -> 9822054.550868498
 12 m5d.xlarge costing 8940.17 -> 9822480.597882776
 12 m5d.xlarge costing 8939.92 -> 9822708.437667957
 12 m5d.xlarge costing 8939.92 -> 9822708.437667957
 12 m5d.xlarge costing 8939.66 -> 9822947.316795923
 12 m5d.xlarge costing 8939.66 -> 9822947.316795923
 12 m5d.xlarge costing 8939.16 -> 9823447.815628342
 12 m5d.xlarge costing 8938.91 -> 9823706.671307586
 12 m5d.xlarge costing 8938.91 -> 9823706.671307586
 12 m5d.xlarge costing 8938.91 -> 9823706.671307586
 12 m5d.xlarge costing 8938.91 -> 9823706.671307586
 12 m5d.xlarge costing 8938.66 -> 9823986.51668122
 12 m5d.xlarge costing 8938.66 -> 9823986.51668122
 12 m5d.xlarge costing 8938.40 -> 9824282.221803896
 12 m5d.xlarge costing 8938.40 -> 9824282.221803896
 12 m5d.xlarge costing 8938.40 -> 9824282.221803896
 12 m5d.xlarge